# Lesson 3 - Building a QA Agent with Claude on Vertex AI

In this lesson, you will build a basic Question Answering (QA) agent using Anthropic's Claude model on Vertex AI. You will start by directly interacting with the model to analyze a PDF document containing health insurance policy details. Then, you will refactor this logic into a reusable Python class, laying the groundwork for the next lesson where you will wrap this agent in an Agent2Agent (A2A) server.

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To Access the <code>requirements.txt</code> and <code>helpers</code> files, and the <code>data</code> folder:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. The PDF document is in the <code>data</code> folder.</p> 

## 3.1. Import Libraries and Setup

First, import the necessary libraries. You will use `AnthropicVertex` to interact with the LLM and standard libraries to handle file encoding.

In [ ]:
import base64
from pathlib import Path

from IPython.display import Markdown, display
from anthropic import AnthropicVertex
from anthropic.types import (
    Base64PDFSourceParam,
    DocumentBlockParam,
    MessageParam,
    TextBlockParam,
)
from dotenv import load_dotenv

from helpers import authenticate

In [ ]:
_ = load_dotenv()

You can see an example of the environment variables defined [here](https://github.com/holtskinner/A2AWalkthrough/blob/main/example.env).

## 3.2. Initialize the Client and Load Data

Here you initialize the `AnthropicVertex` client using your project credentials.


In [ ]:
credentials, project_id = authenticate()

**Note:** When using models through Vertex AI, there's more than one way to authenticate. The course materials use a service account key for authentication. You can refer to this [guide](https://docs.cloud.google.com/docs/authentication/provide-credentials-adc) to learn more about other authentication methods. If you'd like to run the course materials locally, make sure to refer to the [course repo](https://github.com/https-deeplearning-ai/sc-google-a2a-c8-platform#).

In [ ]:
client = AnthropicVertex(
    project_id=project_id,
    region="global",
    access_token=credentials.token,
)

You also read the insurance policy PDF (`2026AnthemgHIPSBC.pdf`) and encode it in base64 so it can be passed to the model as context. You can find the PDF document in the [course repo](https://github.com/holtskinner/A2AWalkthrough/tree/main/data) or in the `data` folder (please check the note at the beginning of the notebook on how to access the `data` folder).

In [ ]:
with Path("../data/2026AnthemgHIPSBC.pdf").open("rb") as file:
    pdf_data = base64.standard_b64encode(file.read()).decode("utf-8")

## 3.3. Query the Model

Now you will send a specific query to the model: "How much would I pay for mental health therapy?". You provide the model with a system instruction to act as an expert insurance agent and pass the PDF document alongside the user's text prompt.

In [ ]:
prompt = "How much would I pay for mental health therapy?"

In [ ]:
response = client.messages.create(
    model="claude-haiku-4-5@20251001",
    max_tokens=1024,
    system="""You are an expert insurance agent designed to assist with 
    coverage queries. Use the provided documents to answer questions 
    about insurance policies. If the information is not available in 
    the documents, respond with 'I don't know'""",
    messages=[
        MessageParam(
            role="user",
            content=[
                DocumentBlockParam(
                    type="document",
                    source=Base64PDFSourceParam(
                        type="base64",
                        media_type="application/pdf",
                        data=pdf_data,
                    ),
                ),
                TextBlockParam(
                    type="text",
                    text=prompt,
                ),
            ],
        )
    ],
)

In [ ]:
response_text = response.content[0].text.replace("$", r"\\$")
display(Markdown(response_text))

## 3.4. Refactor into an Agent Class

To make this code reusable and easier to integrate into an A2A server later, you will wrap the logic into a `PolicyAgent` class in a file named `agents.py`. This class initializes the client and data in the `__init__` method and exposes an `answer_query` method.

In [ ]:
%%writefile agents.py
import base64
from pathlib import Path

from anthropic import AnthropicVertex
from anthropic.types import (
    Base64PDFSourceParam,
    DocumentBlockParam,
    MessageParam,
    TextBlockParam,
)
from dotenv import load_dotenv

from helpers import authenticate

class PolicyAgent:
    def __init__(self) -> None:
        load_dotenv()
        credentials, project_id = authenticate()
        self.client = AnthropicVertex(
            project_id=project_id,
            region="global",
            access_token=credentials.token,
        )
        with Path("../data/2026AnthemgHIPSBC.pdf").open("rb") as file:
            self.pdf_data = base64.standard_b64encode(file.read()).decode("utf-8")

    def answer_query(self, prompt: str) -> str:
        response = self.client.messages.create(
            model="claude-haiku-4-5@20251001",
            max_tokens=1024,
            system="You are an expert insurance agent designed to assist with coverage queries. Use the provided documents to answer questions about insurance policies. If the information is not available in the documents, respond with 'I don't know'",
            messages=[
                MessageParam(
                    role="user",
                    content=[
                        DocumentBlockParam(
                            type="document",
                            source=Base64PDFSourceParam(
                                type="base64",
                                media_type="application/pdf",
                                data=self.pdf_data,
                            ),
                        ),
                        TextBlockParam(
                            type="text",
                            text=prompt,
                        ),
                    ],
                )
            ],
        )
        
        return response.content[0].text.replace("$", r"\\$")

## 3.5. Test the Agent Class

Finally, import the `PolicyAgent` class you just created and test it with the same query to ensure it works as expected.

In [ ]:
from agents import PolicyAgent

print("Running Health Insurance Policy Agent")
agent = PolicyAgent()
prompt = "How much would I pay for mental health therapy?"

response = agent.answer_query(prompt)
display(Markdown(response))

## 3.6. Resources

- [Anthropic on Vertex AI](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/partner-models/claude)
- [Course Repo](https://github.com/holtskinner/A2AWalkthrough):
   - It has a slightly different [notebook version](https://github.com/holtskinner/A2AWalkthrough/blob/main/1_BasicQA.ipynb) that uses Gemini instead of Claude

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download"</em>.</p>
</div>
